In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from functools import reduce
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import operator
from sklearn.svm import SVC
import xgboost as xgb
import gensim
from sklearn.cluster import KMeans

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
reviews_with_prod_label = pd.read_csv('reviews_with_prod_label.csv')
print(reviews_with_prod_label.head())
print(reviews_with_prod_label.columns)
print(reviews_with_prod_label.shape)

   Unnamed: 0  index  review_rating  number_of_helpful  \
0           0      0            5.0                0.0   
1           1      1            5.0                2.0   
2           2      2            5.0                0.0   
3           3      3            5.0                0.0   
4           4      4            5.0                0.0   

                                         review_body           review_title  \
0  JUST WHAT I THOUGHT IT WAS AND ORDER WENT SMOO...             Five Stars   
1  After I saw the motion picture, Heaven is for ...  Wallet photo of Jesus   
2  Great quality of image and prompty service, gr...             Five Stars   
3     Loved them gave them out to sunday bible study             Five Stars   
4       Arrived in perfect condition. No complaints.             Five Stars   

  review_date  number_of_photos  product_ID  reviewer_ID  fake  
0  2014-12-14                 0           0      2270578     0  
1  2014-12-13                 0           0   

In [ ]:
'''products_keeping = reviews_with_prod_label["product_ID"].value_counts().head(500)
print(products_keeping)
products_keeping = products_keeping.index.tolist()
reviews_with_prod_label = reviews_with_prod_label[reviews_with_prod_label["product_ID"].isin(products_keeping)]
print(reviews_with_prod_label.shape)
print(reviews_with_prod_label["product_ID"].value_counts())'''

'products_keeping = reviews_with_prod_label["product_ID"].value_counts().head(500)\nprint(products_keeping)\nproducts_keeping = products_keeping.index.tolist()\nreviews_with_prod_label = reviews_with_prod_label[reviews_with_prod_label["product_ID"].isin(products_keeping)]\nprint(reviews_with_prod_label.shape)\nprint(reviews_with_prod_label["product_ID"].value_counts())'

In [ ]:
reviews_with_prod_label['review_date'] = pd.to_datetime(reviews_with_prod_label['review_date'])
reviews_with_prod_label = reviews_with_prod_label.sort_values(['product_ID', 'review_date'])

reviews_with_prod_label['days_between_reviews'] = (
    reviews_with_prod_label.groupby('product_ID')['review_date']
      .diff()
      .dt.days
)
reviews_with_prod_label['days_between_reviews'] = reviews_with_prod_label['days_between_reviews'].fillna(0)

reviews_with_prod_label["text"] = reviews_with_prod_label["review_title"].fillna("") + " " + reviews_with_prod_label["review_body"].fillna("")
reviews_with_prod_label["text"] = reviews_with_prod_label["text"].apply(lambda x: re.sub(r"[^a-zA-Z\s]", "", x.lower()))


print(reviews_with_prod_label.head())
print(reviews_with_prod_label.columns)
print(reviews_with_prod_label.shape)

   Unnamed: 0  index  review_rating  number_of_helpful  \
9           9      9            5.0                2.0   
8           8      8            5.0                0.0   
7           7      7            5.0                0.0   
6           6      6            5.0                0.0   
5           5      5            5.0                0.0   

                                         review_body  \
9  Beautiful wallet cards.  I have been sharing t...   
8         I love them and will order more very soon.   
7                        As described; fast delivery   
6  Exactly what I thought it would be. Love the p...   
5  I hand these wonderful Prince of Peace cards o...   

                  review_title review_date  number_of_photos  product_ID  \
9      Beautiful Artist Design  2014-09-16                 0           0   
8                   Five Stars  2014-09-20                 0           0   
7                   Five Stars  2014-09-28                 0           0   
6  Love th

In [ ]:
reviews_metadata = reviews_with_prod_label.groupby(["product_ID"]).agg(
    n_of_reviews = ('index', 'count'),
    avg_review_rating = ('review_rating', 'mean'),
    avg_days_between_reviews = ('days_between_reviews', 'mean'),
    stdev_days_between_reviews = ('days_between_reviews', 'std'),
    max_days_between_reviews = ('days_between_reviews', 'max'),
    min_days_between_reviews = ('days_between_reviews', 'min'),
    avg_helpful_reviews = ('number_of_helpful', 'mean'),
    avg_1star_reviews = ('review_rating', lambda x: (x == 1).mean()),
    avg_5star_reviews = ('review_rating', lambda x: (x == 5).mean()),
    avg_photo_reviews = ('number_of_photos', 'mean'),
    std_review_len = ('text', lambda x: x.apply(lambda y: len(y.split())).std())
)
reviews_metadata = reviews_metadata.reset_index()
reviews_metadata = reviews_metadata.merge(reviews_with_prod_label, on='product_ID', how='left')
reviews_metadata = reviews_metadata.drop_duplicates(subset=['product_ID'])
reviews_metadata = reviews_metadata[['product_ID', 'n_of_reviews', 'avg_review_rating', 'avg_days_between_reviews', 'stdev_days_between_reviews', 'max_days_between_reviews', 'min_days_between_reviews', 'avg_helpful_reviews', 'avg_1star_reviews', 'avg_5star_reviews', 'avg_photo_reviews', 'std_review_len', 'index', 'review_rating', 'number_of_helpful', 'review_body', 'review_title', 'review_date', 'number_of_photos', 'reviewer_ID', 'fake']]

print(reviews_metadata.head())
print(reviews_metadata.columns)
print(reviews_metadata.shape)

     product_ID  n_of_reviews  avg_review_rating  avg_days_between_reviews  \
0             0            89           4.932584                 14.696629   
89            1            52           4.673077                 47.134615   
141           2           123           4.130081                 33.471545   
264           3           176           4.460227                 23.022727   
440           4           455           4.450549                  5.859341   

     stdev_days_between_reviews  max_days_between_reviews  \
0                     16.789746                      72.0   
89                    64.125499                     347.0   
141                   80.160630                     848.0   
264                   75.075730                     886.0   
440                    9.553812                     102.0   

     min_days_between_reviews  avg_helpful_reviews  avg_1star_reviews  \
0                         0.0             0.337079           0.000000   
89                

In [ ]:
pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

data = []
for i in reviews_with_prod_label["text"]:
  text = pattern.sub('', i)
  data.append(text)

reviews_with_prod_label["text"] = data

stemmer = SnowballStemmer(language='english')
reviews_with_prod_label["text"] = reviews_with_prod_label["text"].apply(lambda x: reduce(lambda y, z: y + " " + stemmer.stem(z), word_tokenize(x), ""))

print(reviews_with_prod_label.head())
print(reviews_with_prod_label.columns)
print(reviews_with_prod_label.shape)

   Unnamed: 0  index  review_rating  number_of_helpful  \
9           9      9            5.0                2.0   
8           8      8            5.0                0.0   
7           7      7            5.0                0.0   
6           6      6            5.0                0.0   
5           5      5            5.0                0.0   

                                         review_body  \
9  Beautiful wallet cards.  I have been sharing t...   
8         I love them and will order more very soon.   
7                        As described; fast delivery   
6  Exactly what I thought it would be. Love the p...   
5  I hand these wonderful Prince of Peace cards o...   

                  review_title review_date  number_of_photos  product_ID  \
9      Beautiful Artist Design  2014-09-16                 0           0   
8                   Five Stars  2014-09-20                 0           0   
7                   Five Stars  2014-09-28                 0           0   
6  Love th

In [ ]:
X = reviews_metadata[["product_ID"]]
y = reviews_metadata[["fake", "product_ID"]]

X_train, X_rest, y_train, y_rest  = train_test_split(X, y, train_size=0.8, random_state = 42, stratify = y["fake"])
X_validation, X_test, y_validation, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state = 42, stratify=y_rest['fake'])

print(X_train.shape)
print(X_test.shape)
print(X_validation.shape)

(2726, 1)
(341, 1)
(341, 1)


In [ ]:
X_train_text = reviews_with_prod_label[reviews_with_prod_label["product_ID"].isin(X_train["product_ID"])]
X_validation_text = reviews_with_prod_label[reviews_with_prod_label["product_ID"].isin(X_validation["product_ID"])]
X_test_text = reviews_with_prod_label[reviews_with_prod_label["product_ID"].isin(X_test["product_ID"])]

print(X_train_text.shape)
print(X_validation_text.shape)
print(X_test_text.shape)
print(X_train_text.head())

(570169, 13)
(73840, 13)
(70547, 13)
   Unnamed: 0  index  review_rating  number_of_helpful  \
9           9      9            5.0                2.0   
8           8      8            5.0                0.0   
7           7      7            5.0                0.0   
6           6      6            5.0                0.0   
5           5      5            5.0                0.0   

                                         review_body  \
9  Beautiful wallet cards.  I have been sharing t...   
8         I love them and will order more very soon.   
7                        As described; fast delivery   
6  Exactly what I thought it would be. Love the p...   
5  I hand these wonderful Prince of Peace cards o...   

                  review_title review_date  number_of_photos  product_ID  \
9      Beautiful Artist Design  2014-09-16                 0           0   
8                   Five Stars  2014-09-20                 0           0   
7                   Five Stars  2014-09-28       

In [ ]:
X_train_text_combined = (
    X_train_text.groupby('product_ID')['text']
    .apply(lambda x: ' '.join(x))
    .reset_index()
)

print(X_train_text_combined.head())
print(X_train_text_combined.shape)
print(X_train_text_combined.columns)

   product_ID                                               text
0           0   beauti artist design beauti wallet card share...
1           2   magnet poetri kid spanish kit kit fun way kid...
2           3   great littl gift colleg magnet poetri kit nea...
3           4   littl pack punch addit exhaust current collec...
4           5   rip deserv zero star item arriv plastic wrap ...
(2726, 2)
Index(['product_ID', 'text'], dtype='object')


In [ ]:
X_validation_text_combined = (
    X_validation_text.groupby('product_ID')['text']
    .apply(lambda x: ' '.join(x))
    .reset_index()
)

print(X_validation_text_combined.head())
print(X_validation_text_combined.shape)
print(X_validation_text_combined.columns)

   product_ID                                               text
0          26   great blender respect ive blender year use re...
1          98   oxo good alway oxo grood grip nylon squar tur...
2         122   bang buck first thing notic tong got weight i...
3         123   great aa meet great coffe urn larg aa meet al...
4         144   great knife may want larger chef knife hencke...
(341, 2)
Index(['product_ID', 'text'], dtype='object')


In [ ]:
X_test_text_combined = (
    X_test_text.groupby('product_ID')['text']
    .apply(lambda x: ' '.join(x))
    .reset_index()
)

print(X_test_text_combined.head())
print(X_test_text_combined.shape)
print(X_test_text_combined.columns)

   product_ID                                               text
0           1   speechless look akian book speechless everi p...
1           8   greatest corkscrew easiest corkscrew maneuv t...
2          13   tupperwar orang peeler fantast couldnt enjoy ...
3          14   what love serious doesnt love tupperwar bowl ...
4          20   corkscrew plant real look bought two glad pri...
(341, 2)
Index(['product_ID', 'text'], dtype='object')


In [ ]:
tfidf = TfidfVectorizer(max_features=1000)
X_train_text_tfidf = tfidf.fit_transform(X_train_text_combined["text"])
X_validation_text_tfidf = tfidf.transform(X_validation_text_combined["text"])
X_test_text_tfidf = tfidf.transform(X_test_text_combined["text"])
tfidf_features = tfidf.get_feature_names_out()

X_train_text_tfidf = X_train_text_tfidf.toarray()
X_validation_text_tfidf = X_validation_text_tfidf.toarray()
X_test_text_tfidf = X_test_text_tfidf.toarray()

scaler = StandardScaler()
X_train_text_tfidf_scaled = scaler.fit_transform(X_train_text_tfidf)
X_validation_text_tfidf_scaled = scaler.transform(X_validation_text_tfidf)
X_test_text_tfidf_scaled = scaler.transform(X_test_text_tfidf)


X_train_tfidf_scaled = pd.DataFrame(X_train_text_tfidf_scaled, columns = tfidf_features)
X_validation_tfidf_scaled = pd.DataFrame(X_validation_text_tfidf_scaled, columns = tfidf_features)
X_test_tfidf_scaled = pd.DataFrame(X_test_text_tfidf_scaled, columns = tfidf_features)

print(X_train_tfidf_scaled.head())
print(X_train_tfidf_scaled.shape)
print(X_validation_tfidf_scaled.shape)
print(X_test_tfidf_scaled.shape)

        abl   absolut     accur    across    actual        ad       add  \
0  7.389573  0.657631 -0.201412 -0.558995  0.094573  3.310226  2.007754   
1  0.892930  0.209545  0.852400  0.615070  0.147080 -0.727027  0.715906   
2  0.178094  0.052885 -0.201412  0.434037 -0.435393  0.457800  1.290315   
3 -0.359568 -0.441370 -0.201412 -0.137110 -0.708560  0.531394  1.497053   
4 -1.062074  1.035363 -0.201412 -0.558995 -0.381439 -0.727027 -0.686187   

      addit    adjust  advertis  ...     would   wouldnt      wrap     write  \
0 -0.606606 -0.295983 -0.789049  ... -1.240810 -0.951741 -0.229543 -0.392328   
1  0.493250 -0.295983 -0.789049  ...  0.936067 -0.223426 -0.229543  0.571524   
2  0.633749 -0.295983  0.437156  ...  0.594981  0.280288 -0.229543  6.944769   
3  3.740805 -0.295983 -0.615400  ... -0.108069 -0.690030 -0.229543  2.378456   
4  0.058004 -0.295983  0.963025  ...  0.778664  1.688855  0.322591 -0.392328   

      wrong      year       yes       yet     youll      your  
0 -0

In [ ]:
X_train_tfidf_avg = np.array(X_train_text_tfidf.mean(axis=1)).ravel()
X_validation_tfidf_avg = np.array(X_validation_text_tfidf.mean(axis=1)).ravel()
X_test_tfidf_avg = np.array(X_test_text_tfidf.mean(axis=1)).ravel()

X_train_text_combined['tfidf_review_body'] = X_train_tfidf_avg
X_validation_text_combined['tfidf_review_body'] = X_validation_tfidf_avg
X_test_text_combined['tfidf_review_body'] = X_test_tfidf_avg

print(X_train_text_combined.head())
print(X_train_text_combined.shape)
print(X_validation_text_combined.shape)
print(X_test_text_combined.shape)

   product_ID                                               text  \
0           0   beauti artist design beauti wallet card share...   
1           2   magnet poetri kid spanish kit kit fun way kid...   
2           3   great littl gift colleg magnet poetri kit nea...   
3           4   littl pack punch addit exhaust current collec...   
4           5   rip deserv zero star item arriv plastic wrap ...   

   tfidf_review_body  
0           0.008585  
1           0.007897  
2           0.007970  
3           0.007795  
4           0.010168  
(2726, 3)
(341, 3)
(341, 3)


In [ ]:
def weighted_projected_graph(B, nodes, ratio=False):
  if B.is_directed():
    pred = B.pred
    G = nx.DiGraph()
  else:
    pred = B.adj
    G = nx.Graph()
  G.graph.update(B.graph)
  G.add_nodes_from((n, B.nodes[n]) for n in nodes)
  n_top = float(len(B) - len(nodes))
  nodes_checked = []
  for u in nodes:
    nodes_checked.append(u)
    unbrs = set(B[u])
    nbrs2 = {n for nbr in unbrs for n in B[nbr]} - set(nodes_checked)
    for v in nbrs2:
      vnbrs = set(pred[v])
      common = unbrs & vnbrs
      if not ratio:
        weight = len(common)
      else:
        weight = len(common) / n_top
      G.add_edge(u, v, weight=weight)
  return G

def obtain_network_features(reviews):
  # initializing the product-level data
  df = pd.DataFrame({"product_ID": reviews.product_ID.unique()})

  # building the bipartite product-reviewer graph
  B = nx.Graph()
  B.add_nodes_from(reviews.reviewer_ID, bipartite=0)
  B.add_nodes_from(reviews.product_ID, bipartite=1)
  B.add_edges_from([(row['reviewer_ID'], row['product_ID']) for idx, row in reviews.iterrows()])

  # building the product projected graph
  P = weighted_projected_graph(B, reviews.product_ID.unique())

  w_degree_cent = nx.degree(P, weight='weight')
  eig_cent = nx.eigenvector_centrality(P, max_iter=500)
  pr = nx.pagerank(P, alpha=0.85)
  cc = nx.clustering(P)
  w_cc = nx.clustering(P, weight='weight')
  w_eig_cent = nx.eigenvector_centrality(P, max_iter=500, weight = "weight")
  degree = nx.degree(P)
  w_pr = nx.pagerank(P, alpha=0.85, weight='weight')
  degree = nx.degree(P)

  # creating the features data
  df['pagerank'] = [pr[i] for i in df.product_ID]
  df['eigenvector_cent'] = [eig_cent[i] for i in df.product_ID]
  df['clustering_coef'] = [cc[i] for i in df.product_ID]
  df['w_degree'] = [w_degree_cent[i] for i in df.product_ID]
  df['degree'] = [degree[i] for i in df.product_ID]
  df['w_pagerank'] = [w_pr[i] for i in df.product_ID]
  df['w_clustering_coef'] = [w_cc[i] for i in df.product_ID]
  df['w_eigenvector_cent'] = [w_eig_cent[i] for i in df.product_ID]
  return df


X_train_network = obtain_network_features(X_train_text)
X_validation_network = obtain_network_features(X_validation_text)
X_test_network = obtain_network_features(X_test_text)

X_train_with_network = X_train_network.merge(X_train_text, on='product_ID', how='inner')
X_validation_with_network = X_validation_network.merge(X_validation_text, on='product_ID', how='inner')
X_test_with_network = X_test_network.merge(X_test_text, on='product_ID', how='inner')

print(X_train_with_network.shape)
print(X_validation_with_network.shape)
print(X_test_with_network.shape)

(570169, 21)
(73840, 21)
(70547, 21)


In [ ]:
X_train_with_network = X_train_with_network.drop_duplicates(subset=['product_ID'])
X_validation_with_network = X_validation_with_network.drop_duplicates(subset=['product_ID'])
X_test_with_network = X_test_with_network.drop_duplicates(subset=['product_ID'])

print(X_train_with_network.head())
print(X_train_with_network.columns)
print(X_train_with_network.shape)

     product_ID  pagerank  eigenvector_cent  clustering_coef  w_degree  \
0             0  0.000006          0.000565         0.200000         5   
89            2  0.000171          0.003374         0.012987        78   
212           3  0.000033          0.002529         0.052632        35   
388           4  0.000098          0.014515         0.134047        95   
843           5  0.001929          0.002821         0.001440       775   

     degree  w_pagerank  w_clustering_coef  w_eigenvector_cent  Unnamed: 0  \
0         5    0.000006           0.000227            0.000002           9   
89       78    0.000171           0.000020            0.000023         123   
212      19    0.000033           0.000095            0.000011         239   
388      79    0.000098           0.000162            0.000060         406   
843     775    0.001929           0.000002            0.000007         854   

     ...  review_rating  number_of_helpful  \
0    ...            5.0                2

In [ ]:
network_features = ['pagerank', 'w_degree', 'clustering_coef', 'eigenvector_cent']

review_features = ['tfidf_review_body', 'n_of_reviews','avg_review_rating',
                   'avg_days_between_reviews', 'stdev_days_between_reviews',
                   'max_days_between_reviews', 'min_days_between_reviews',
                   'avg_helpful_reviews', 'avg_1star_reviews', 'avg_5star_reviews', 'avg_photo_reviews', 'std_review_len']

In [ ]:
X_train_metadata = reviews_metadata[reviews_metadata["product_ID"].isin(X_train["product_ID"])]
X_validation_metadata = reviews_metadata[reviews_metadata["product_ID"].isin(X_validation["product_ID"])]
X_test_metadata = reviews_metadata[reviews_metadata["product_ID"].isin(X_test["product_ID"])]

X_train_metadata = pd.merge(X_train_metadata, X_train_text_combined, on='product_ID', how='inner')
X_validation_metadata = pd.merge(X_validation_metadata, X_validation_text_combined, on='product_ID', how='inner')
X_test_metadata = pd.merge(X_test_metadata, X_test_text_combined, on='product_ID', how='inner')

X_train_all_features = pd.merge(X_train_metadata, X_train_with_network, on='product_ID', how='inner')
X_validation_all_features = pd.merge(X_validation_metadata, X_validation_with_network, on='product_ID', how='inner')
X_test_all_features = pd.merge(X_test_metadata, X_test_network, on='product_ID', how='inner')

X_train_features_to_scale = X_train_all_features[network_features + review_features]
X_validation_features_to_scale = X_validation_all_features[network_features + review_features]
X_test_features_to_scale = X_test_all_features[network_features + review_features]

scaler = StandardScaler()
X_train_all_features_scaled = scaler.fit_transform(X_train_features_to_scale)
X_validation_all_features_scaled = scaler.transform(X_validation_features_to_scale)
X_test_all_features_scaled = scaler.transform(X_test_features_to_scale)

X_train_all_features_scaled = pd.DataFrame(X_train_all_features_scaled, columns = network_features + review_features)
X_validation_all_features_scaled = pd.DataFrame(X_validation_all_features_scaled, columns = network_features + review_features)
X_test_all_features_scaled = pd.DataFrame(X_test_all_features_scaled, columns = network_features + review_features)

X_train_all_features_scaled["product_ID"] = X_train_all_features["product_ID"]
X_validation_all_features_scaled["product_ID"] = X_validation_all_features["product_ID"]
X_test_all_features_scaled["product_ID"] = X_test_all_features["product_ID"]

print(X_train_all_features_scaled.head())
print(X_train_all_features_scaled.shape)
print(X_train_all_features_scaled.columns)
print(X_validation_all_features_scaled.shape)
print(X_test_all_features_scaled.shape)

   pagerank  w_degree  clustering_coef  eigenvector_cent  tfidf_review_body  \
0 -0.429656 -0.630363         0.663855         -0.897894           0.212843   
1 -0.075096 -0.246504        -1.436008         -0.696249          -0.108062   
2 -0.370769 -0.472613        -0.990862         -0.756914          -0.074254   
3 -0.231531 -0.157112        -0.076689          0.103389          -0.155849   
4  3.705905  3.418564        -1.565659         -0.735922           0.951707   

   n_of_reviews  avg_review_rating  avg_days_between_reviews  \
0     -0.627134           1.528118                 -0.859519   
1     -0.449682          -0.023405                 -0.027106   
2     -0.173066           0.614884                 -0.490369   
3      1.283084           0.596174                 -1.251333   
4     -0.522750           0.310429                 -0.662215   

   stdev_days_between_reviews  max_days_between_reviews  \
0                   -1.000201                 -1.131554   
1                    0

In [ ]:
y_train_labels = y_train[["fake"]]
y_validation_labels = y_validation[["fake"]]
y_test_labels = y_test[["fake"]]

y_train_copy = y_train_labels.values.ravel()
y_validation_copy = y_validation_labels.values.ravel()
y_test_copy = y_test_labels.values.ravel()
print(y_train_copy)
print(y_train_copy.shape)
print(y_validation_copy.shape)
print(y_test_copy.shape)

[1 0 0 ... 0 0 0]
(2726,)
(341,)
(341,)


##Supervised

In [ ]:
def model_building(X_train, y_train, X_test, y_test, model):

	model.fit(X_train, y_train)

	y_pred = model.predict(X_test)
	cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
	probs = model.predict_proba(X_test)[:,1]

	# print(cm)
	print("AUC, Accuracy, TN, TP, F1 Score")
	print("{}, {}, {}, {}, {}".format(metrics.roc_auc_score(y_test, model.predict_proba(X_test)[:,1]),
															  sum(cm.diagonal()) / X_test.shape[0],
															  cm[0,0] / sum(cm[0,:]),
															  cm[1,1] / sum(cm[1,:]),
															  metrics.f1_score(y_test, y_pred, average='weighted')))

	return probs

In [ ]:
def classification_results(X_train, X_validation, X_test, features=None):
  X_train = X_train[features]
  X_validation = X_validation[features]
  X_test = X_test[features]

  print("="*10 + "Logistic Regression" + "="*10)
  model = LogisticRegression(max_iter=400)
  model_building(X_train, y_train_copy, X_validation, y_validation_copy, model)

  print("="*10 + "Random Forest" + "="*10)
  model = RandomForestClassifier(random_state=42,
                                 n_estimators=100,
	                               min_samples_leaf=3,
	                               min_samples_split=6,
	                               max_features='sqrt',
	                               max_depth=40,
	                               bootstrap=True,
	                               n_jobs=-1)

  model_building(X_train, y_train_copy, X_validation, y_validation_copy, model)

  print("="*10 + "RF Feature Importance" + "="*10)
  imps = model.feature_importances_
  feat_imp = {features[i]: imps[i] for i in range(len(features))}

  if len(features) > 100:
    print(sorted(feat_imp.items(), key=operator.itemgetter(1), reverse=True)[:50])
  else:
    print(sorted(feat_imp.items(), key=operator.itemgetter(1), reverse=True))

  print("="*10 + "SVC Linear" + "="*10)
  model = SVC(kernel='linear', probability=True)
  model_building(X_train, y_train_copy, X_validation, y_validation_copy, model)

  print("="*10 + "XGBoost" + "="*10)
  model = xgb.XGBClassifier()
  model_building(X_train, y_train_copy, X_validation, y_validation_copy, model)

  return

In [ ]:
classification_results(X_train_all_features_scaled, X_validation_all_features_scaled, X_test_all_features_scaled, features=review_features)

==========Logistic Regression==========
AUC, Accuracy, TN, TP, F1 Score
0.46489936180657826, 0.5425219941348973, 0.9329896907216495, 0.027210884353741496, 0.41861003308682304
==========Random Forest==========
AUC, Accuracy, TN, TP, F1 Score
0.4820814923907708, 0.5073313782991202, 0.7422680412371134, 0.19727891156462585, 0.4699471539773836
==========RF Feature Importance==========
[('std_review_len', np.float64(0.10257260769451895)), ('avg_helpful_reviews', np.float64(0.10108611683250172)), ('tfidf_review_body', np.float64(0.096262445746845)), ('avg_5star_reviews', np.float64(0.09551615118743419)), ('avg_days_between_reviews', np.float64(0.09497650467564038)), ('avg_1star_reviews', np.float64(0.09162484307213858)), ('max_days_between_reviews', np.float64(0.0916116174387369)), ('stdev_days_between_reviews', np.float64(0.08989452602849551)), ('avg_review_rating', np.float64(0.08731106106186344)), ('n_of_reviews', np.float64(0.08447560745393888)), ('avg_photo_reviews', np.float64(0.0646685

In [ ]:
classification_results(X_train_all_features_scaled, X_validation_all_features_scaled, X_test_all_features_scaled, features=network_features)

==========Logistic Regression==========
AUC, Accuracy, TN, TP, F1 Score
0.5436566379128971, 0.5747800586510264, 0.9948453608247423, 0.02040816326530612, 0.43069073815799674
==========Random Forest==========
AUC, Accuracy, TN, TP, F1 Score
0.507135844028333, 0.501466275659824, 0.4742268041237113, 0.5374149659863946, 0.5033640287444179
==========RF Feature Importance==========
[('pagerank', np.float64(0.2755891157069113)), ('eigenvector_cent', np.float64(0.27118006707847553)), ('clustering_coef', np.float64(0.2645228501880632)), ('w_degree', np.float64(0.18870796702654982))]
==========SVC Linear==========
AUC, Accuracy, TN, TP, F1 Score
0.476225541763097, 0.5689149560117303, 1.0, 0.0, 0.4125962671636474
==========XGBoost==========
AUC, Accuracy, TN, TP, F1 Score
0.512430745494074, 0.46920821114369504, 0.22164948453608246, 0.7959183673469388, 0.42631565505489616


In [ ]:
classification_results(X_train_all_features_scaled, X_validation_all_features_scaled, X_test_all_features_scaled, features=['eigenvector_cent', 'clustering_coef'])

==========Logistic Regression==========
AUC, Accuracy, TN, TP, F1 Score
0.4911284101269373, 0.5689149560117303, 1.0, 0.0, 0.4125962671636474
==========Random Forest==========
AUC, Accuracy, TN, TP, F1 Score
0.5113963111017602, 0.5454545454545454, 0.7474226804123711, 0.2789115646258503, 0.5199053542105739
==========RF Feature Importance==========
[('eigenvector_cent', np.float64(0.5058214220511374)), ('clustering_coef', np.float64(0.49417857794886255))]
==========SVC Linear==========
AUC, Accuracy, TN, TP, F1 Score
0.507749491549197, 0.5689149560117303, 1.0, 0.0, 0.4125962671636474
==========XGBoost==========
AUC, Accuracy, TN, TP, F1 Score
0.5061540079949506, 0.5366568914956011, 0.7525773195876289, 0.25170068027210885, 0.5066638577094639


In [ ]:
classification_results(X_train_all_features_scaled, X_validation_all_features_scaled, X_test_all_features_scaled, features=review_features + network_features)

==========Logistic Regression==========
AUC, Accuracy, TN, TP, F1 Score
0.45308226383336836, 0.5483870967741935, 0.9381443298969072, 0.034013605442176874, 0.42606375061135093
==========Random Forest==========
AUC, Accuracy, TN, TP, F1 Score
0.49179465600673256, 0.5659824046920822, 0.9381443298969072, 0.07482993197278912, 0.46025045282042437
==========RF Feature Importance==========
[('avg_helpful_reviews', np.float64(0.07633057781300394)), ('std_review_len', np.float64(0.07614900978852775)), ('tfidf_review_body', np.float64(0.07247745303523534)), ('avg_5star_reviews', np.float64(0.07211723779973701)), ('clustering_coef', np.float64(0.0716020057127603)), ('avg_1star_reviews', np.float64(0.07154069854539177)), ('max_days_between_reviews', np.float64(0.07031940142264377)), ('avg_review_rating', np.float64(0.06648851420904746)), ('avg_days_between_reviews', np.float64(0.06595418207076773)), ('stdev_days_between_reviews', np.float64(0.06574580551057345)), ('eigenvector_cent', np.float64(0.0

In [ ]:
classification_results(X_train_tfidf_scaled, X_validation_tfidf_scaled, X_test_tfidf_scaled, features = tfidf_features)

==========Logistic Regression==========
AUC, Accuracy, TN, TP, F1 Score
0.5131846553054211, 0.5454545454545454, 0.6185567010309279, 0.4489795918367347, 0.5439389261556598
==========Random Forest==========
AUC, Accuracy, TN, TP, F1 Score
0.5035591556210113, 0.5571847507331378, 0.8814432989690721, 0.1292517006802721, 0.48133629694841495
==========RF Feature Importance==========
[('look', np.float64(0.003060593321960133)), ('find', np.float64(0.002742544249843852)), ('anyon', np.float64(0.0026889324606838043)), ('much', np.float64(0.002413404438442394)), ('old', np.float64(0.0023854888277760255)), ('howev', np.float64(0.0023843247499317245)), ('dont', np.float64(0.002371101037983533)), ('well', np.float64(0.002341862768546941)), ('take', np.float64(0.002245097570469965)), ('made', np.float64(0.002144387985159359)), ('nice', np.float64(0.002135322895602326)), ('worth', np.float64(0.0021139391517956928)), ('disappoint', np.float64(0.0021008274937185257)), ('alreadi', np.float64(0.0020738161

# Clustering - Into Supervised Learning

In [ ]:
k_clusters = 5
kmeans = KMeans(n_clusters=k_clusters, random_state=42)
kmeans.fit(X_train_all_features_scaled)

train_labels = kmeans.labels_
validation_labels = kmeans.predict(X_validation_all_features_scaled)

accuracy_score_lst = []
precision_lst = []
recall_lst = []
f1_lst = []
auc_lst = []

for i in range(k_clusters):
  print("Cluster", i)
  X_train_cluster = X_train_all_features_scaled[train_labels == i]
  X_validation_cluster = X_validation_all_features_scaled[validation_labels == i]

  y_train_cluster = y_train[y_train['product_ID'].isin(X_train_cluster['product_ID'])]
  y_validation_cluster = y_validation[y_validation['product_ID'].isin(X_validation_cluster['product_ID'])]


  y_train_cluster = y_train_cluster[['fake']]
  y_validation_cluster = y_validation_cluster[['fake']]

  y_train_cluster = y_train_cluster.values.ravel()
  y_validation_cluster = y_validation_cluster.values.ravel()

  model = RandomForestClassifier(random_state=42,
	                               n_estimators=1200,
	                               min_samples_leaf=3,
	                               min_samples_split=6,
	                               max_features='sqrt',
	                               max_depth=40,
	                               bootstrap=True,
	                               n_jobs=-1)

  model.fit(X_train_cluster, y_train_cluster)
  y_pred = model.predict(X_validation_cluster)

  accuracy_score = metrics.accuracy_score(y_validation_cluster, y_pred)
  accuracy_score_lst.append(accuracy_score)
  print("Accuracy", accuracy_score)
  precision = metrics.precision_score(y_validation_cluster, y_pred)
  precision_lst.append(precision)
  print("Precision", precision)
  recall = metrics.recall_score(y_validation_cluster, y_pred)
  recall_lst.append(recall)
  print("Recall", recall)
  f1 = metrics.f1_score(y_validation_cluster, y_pred)
  f1_lst.append(f1)
  print("F1 Score", f1)
  auc = metrics.roc_auc_score(y_validation_cluster, model.predict_proba(X_validation_cluster)[:,1])
  auc_lst.append(auc)
  print("AUC", auc)
  print("***********")
  print("\n")


print("\n")
print("Average Across all Clusters")
average_accuracy = sum(accuracy_score_lst) / len(accuracy_score_lst)
print("Accuracy", average_accuracy)
average_precision = sum(precision_lst) / len(precision_lst)
print("Precision", average_precision)
average_recall = sum(recall_lst) / len(recall_lst)
print("Recall", average_recall)
average_f1 = sum(f1_lst) / len(f1_lst)
print("F1 Score", average_f1)
average_auc = sum(auc_lst) / len(auc_lst)
print("AUC", average_auc)

Cluster 0
Accuracy 0.582089552238806
Precision 0.25
Recall 0.13636363636363635
F1 Score 0.17647058823529413
AUC 0.502020202020202
***********


Cluster 1
Accuracy 0.8860759493670886
Precision 0.8860759493670886
Recall 1.0
F1 Score 0.9395973154362416
AUC 0.6047619047619047
***********


Cluster 2
Accuracy 0.9473684210526315
Precision 0.0
Recall 0.0
F1 Score 0.0


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC 0.30246913580246915
***********


Cluster 3
Accuracy 0.5362318840579711
Precision 0.5441176470588235
Recall 0.9736842105263158
F1 Score 0.6981132075471698
AUC 0.38030560271646857
***********


Cluster 4
Accuracy 0.7971014492753623
Precision 0.0
Recall 0.0
F1 Score 0.0


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC 0.32987012987012987
***********




Average Across all Clusters
Accuracy 0.7497734511983719
Precision 0.3360387192851824
Recall 0.42200956937799045
F1 Score 0.36283622224374107
AUC 0.42388539503423484
